In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
# Define the BERT tokenizer and model
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import os
from datetime import datetime
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
name_bert = "alvaroalon2/biobert_diseases_ner"
tokenizer = BertTokenizer.from_pretrained(name_bert)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
path_data_train = "/kaggle/input/iir-semeval-2013-task-9/train_dataset.csv"
path_data_test = "/kaggle/input/iir-semeval-2013-task-9/test_dataset.csv"
path_data_devel = "/kaggle/input/iir-semeval-2013-task-9/Devel_dataset.csv"
train_df = pd.read_csv(path_data_train, sep=',', encoding='utf-8')
test_df = pd.read_csv(path_data_test, sep=',', encoding='utf-8')
devel_df = pd.read_csv(path_data_devel, sep=',', encoding='utf-8')
le = LabelEncoder()
class_mapping = {
    'false': 0,
    'effect': 1,
    'mechanism': 2,
    'advise': 3,
    'int': 4
}
# Fit and transform the 'pair type' column using the custom mapping
train_df['pair type'] = le.fit_transform(train_df['pair type'].map(class_mapping))
test_df['pair type'] = le.fit_transform(test_df['pair type'].map(class_mapping))
devel_df['pair type'] = le.fit_transform(devel_df['pair type'].map(class_mapping))
# Use train_test_split with stratify
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['pair type'], random_state=42)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

In [3]:
train_df.shape, test_df.shape, devel_df.shape, val_df.shape

((41058, 4), (5716, 4), (2703, 4), (4563, 4))

In [4]:
val_df.to_csv("/kaggle/working/train_dataset_3.csv")

In [5]:
# Tokenize and encode the sentences
class MyDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data.iloc[index]['Full Sentence']
        e1 = self.data.iloc[index]['entity e1']
        e2 = self.data.iloc[index]['entity e2']
        pair_type = self.data.iloc[index]['pair type']
        input_text = f"{e1} [SEP] {e2}"
#         input_text = f"{sentence} [SEP] {e1} [SEP] {e2}"

        encoding = self.tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': input_text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(pair_type, dtype=torch.long)
        }


In [6]:
# create an instance of the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(name_bert)
# set the batch size
batch_size = 400
num_epochs = 150  # Adjust as needed

# set the maximum sequence length
max_len = 30

# create data loaders for the training and validation sets
train_dataset = MyDataset(train_df, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

devel_dataset = MyDataset(devel_df, tokenizer, max_len)
devel_loader = DataLoader(devel_dataset, batch_size=batch_size)

test_dataset = MyDataset(test_df, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

val_dataset = MyDataset(val_df, tokenizer, max_len)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [7]:
class BertSentimentClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BertSentimentClassifier, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout1 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 256)
        self.dropout2 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, num_classes)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout1(pooled_output)
        output = self.fc1(pooled_output)
        output = self.dropout2(output)
        logits = self.fc2(output)

        return logits

In [8]:

num_classes = train_df["pair type"].nunique()
model = BertSentimentClassifier(name_bert, num_classes)
criterion = nn.CrossEntropyLoss()

Some weights of BertModel were not initialized from the model checkpoint at alvaroalon2/biobert_diseases_ner and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import torch.nn.functional as F

def focal_loss(outputs, labels, gamma=2.0, alpha=1.0):
    ce_loss = F.cross_entropy(outputs, labels, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = (alpha * (1 - pt) ** gamma * ce_loss).mean()
    return focal_loss

def train(model, data_loader, optimizer, criterion, device):
    # set the model to train mode
    model.train()

    # initialize the loss, accuracy, precision, recall, and f1_score variables
    total_loss, total_accuracy = 0, 0
    total_precision, total_recall, total_f1 = 0, 0, 0

    # iterate over the data loader
    for data in tqdm(data_loader):
        # move the inputs to the device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # zero the gradient
        optimizer.zero_grad()

        # get the model's predictions
        outputs = model(input_ids, attention_mask)

        # get the focal loss
        loss = focal_loss(outputs, labels)
        total_loss += loss.item()

        # accuracy calculation
        logits = outputs.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        total_accuracy += flat_accuracy(logits, label_ids)

        # calculate precision, recall, and f1 score
        total_precision += precision_score(label_ids, np.argmax(logits, axis=1), average='weighted')
        total_recall += recall_score(label_ids, np.argmax(logits, axis=1), average='weighted')
        total_f1 += f1_score(label_ids, np.argmax(logits, axis=1), average='weighted')

        # perform backpropagation and optimization
        loss.backward()
        optimizer.step()

    # calculate the average loss, accuracy, precision, recall, and f1 score
    avg_loss = total_loss / len(data_loader)
    # avg_accuracy = total_accuracy / len(data_loader)
    avg_precision = total_precision / len(data_loader)
    avg_recall = total_recall / len(data_loader)
    avg_f1 = total_f1 / len(data_loader)

    return avg_loss, avg_precision, avg_recall, avg_f1


In [10]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [11]:
def evaluate(model, data_loader, criterion, device):
    # set the model to eval mode
    model.eval()

    # initialize the loss, accuracy, precision, recall, and f1_score variables
    total_loss = 0
    total_accuracy = 0
    total_precision = 0
    total_recall = 0
    total_f1 = 0

    # iterate over the data loader
    for data in tqdm(data_loader):
        # move the inputs to the device
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        # disable gradient computation
        with torch.no_grad():
            # get the model's predictions
            outputs = model(input_ids, attention_mask)

            # get the loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # calculate the number of correct predictions
            logits = outputs.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            total_accuracy += flat_accuracy(logits, label_ids)

            # calculate precision, recall, and f1 score
            total_precision += precision_score(label_ids, np.argmax(logits, axis=1), average='weighted')
            total_recall += recall_score(label_ids, np.argmax(logits, axis=1), average='weighted')
            total_f1 += f1_score(label_ids, np.argmax(logits, axis=1), average='weighted')

    # calculate the average loss, accuracy, precision, recall, and f1 score
    avg_loss = total_loss / len(data_loader)
    # accuracy = total_accuracy / len(data_loader)
    precision = total_precision / len(data_loader)
    recall = total_recall / len(data_loader)
    f1 = total_f1 / len(data_loader)

    return avg_loss, precision, recall, f1

In [12]:
# Function to calculate F1 score, Precision, Recall
def calculate_metrics(predictions, labels):
    predictions = torch.argmax(predictions, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()

    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')

    return f1, precision, recall

In [13]:
import warnings

# ...

# Ignore warnings
warnings.filterwarnings("ignore")

# Create a folder for logs if it doesn't exist
logs_folder = "logs"
os.makedirs(logs_folder, exist_ok=True)

# Generate a filename based on the current date and time
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_filename = f"{logs_folder}/training_log_BERT2_{current_time}.txt"



# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set the learning rate
lr = 2e-5

# set the number of epochs


# set the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# move the model to the device
model.to(device)

# lists to store metrics
train_losses = []
train_precisions = []
train_recalls = []
train_f1_scores = []

devel_losses = []
devel_precisions = []
devel_recalls = []
devel_f1_scores = []

test_losses = []
test_precisions = []
test_recalls = []
test_f1_scores = []

val_losses = []
val_precisions = []
val_recalls = []
val_f1_scores = []

best_test_loss = float('inf')
best_epoch = 0

# iterate over the epochs
# Set up logging
with open(log_filename, 'a') as log_file:
  for epoch in range(num_epochs):
      start_time = datetime.now()
      print(f'Epoch {epoch + 1}/{num_epochs}')

      # train the model
      train_loss, train_precision, train_recall, train_f1 = train(model, train_loader, optimizer, criterion, device)

      # evaluate the model on the validation set
      val_loss, val_precision, val_recall, val_f1 = evaluate(model, val_loader, criterion, device)

      # evaluate the model on the validation set
      devel_loss, devel_precision, devel_recall, devel_f1 = evaluate(model, devel_loader, criterion, device)

          # evaluate the model on the validation set
      test_loss, test_precision, test_recall, test_f1 = evaluate(model, test_loader, criterion, device)

      # print the detailed metrics
      print(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}')
      print(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')
      print(f'Devel -- Loss: {devel_loss:.4f} | Precision: {devel_precision:.4f} | Recall: {devel_recall:.4f} | F1: {devel_f1:.4f}')
      print(f'Test  -- Loss: {test_loss:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f} | F1: {test_f1:.4f}')

      # log the metrics
      log_file.write(f'Epoch {epoch + 1}/{num_epochs}\n')
      log_file.write(f'Train -- Loss: {train_loss:.4f} | Precision: {train_precision:.4f} | Recall: {train_recall:.4f} | F1: {train_f1:.4f}\n')
      log_file.write(f'Val   -- Loss: {val_loss:.4f} | Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}')
      log_file.write(f'Devel -- Loss: {devel_loss:.4f} | Precision: {devel_precision:.4f} | Recall: {devel_recall:.4f} | F1: {devel_f1:.4f}\n')
      log_file.write(f'Test  -- Loss: {test_loss:.4f} | Precision: {test_precision:.4f} | Recall: {test_recall:.4f} | F1: {test_f1:.4f}\n')
      log_file.flush()

      # Save the best model based on validation loss
      if test_loss < best_test_loss:
          best_test_loss = test_loss
          best_epoch = epoch + 1
          torch.save(model.state_dict(), f'best_model_BERT2.pt')

      # Save the metrics
      train_losses.append(train_loss)
      train_precisions.append(train_precision)
      train_recalls.append(train_recall)
      train_f1_scores.append(train_f1)

      val_losses.append(val_loss)
      val_precisions.append(val_precision)
      val_recalls.append(val_recall)
      val_f1_scores.append(val_f1)

      devel_losses.append(devel_loss)
      devel_precisions.append(devel_precision)
      devel_recalls.append(devel_recall)
      devel_f1_scores.append(devel_f1)

      test_losses.append(test_loss)
      test_precisions.append(test_precision)
      test_recalls.append(test_recall)
      test_f1_scores.append(test_f1)

      # Calculate and print time remaining
      elapsed_time = datetime.now() - start_time
      remaining_time = (num_epochs - epoch - 1) * elapsed_time
      print(f'Time Elapsed: {elapsed_time}, Time Remaining: {remaining_time} \n')
      torch.save(model.state_dict(), f'final_model_BERT2.pt')

# Print the best epoch
print(f"Best Model: Epoch {best_epoch}, Best Validation Loss: {best_test_loss:.4f}")

print(f"Path log file: ", log_filename)



Epoch 1/150


100%|██████████| 15/15 [00:08<00:00,  1.70it/s]


Train -- Loss: 0.3715 | Precision: 0.7258 | Recall: 0.8292 | F1: 0.7642
Val   -- Loss: 0.5904 | Precision: 0.7870 | Recall: 0.8538 | F1: 0.8022
Devel -- Loss: 0.7062 | Precision: 0.6881 | Recall: 0.8184 | F1: 0.7448
Test  -- Loss: 0.7078 | Precision: 0.7075 | Recall: 0.8225 | F1: 0.7556
Time Elapsed: 0:02:31.024564, Time Remaining: 6:15:02.660036 

Epoch 2/150


100%|██████████| 15/15 [00:08<00:00,  1.70it/s]


Train -- Loss: 0.2402 | Precision: 0.8168 | Recall: 0.8572 | F1: 0.8161
Val   -- Loss: 0.5329 | Precision: 0.8537 | Recall: 0.8635 | F1: 0.8451
Devel -- Loss: 0.6993 | Precision: 0.7469 | Recall: 0.7922 | F1: 0.7599
Test  -- Loss: 0.7146 | Precision: 0.7250 | Recall: 0.7978 | F1: 0.7557
Time Elapsed: 0:02:28.658575, Time Remaining: 6:06:41.469100 

Epoch 3/150


...

Train -- Loss: 0.0275 | Precision: 0.9521 | Recall: 0.9516 | F1: 0.9506
Val   -- Loss: 0.2405 | Precision: 0.9214 | Recall: 0.9242 | F1: 0.9217
Devel -- Loss: 0.8618 | Precision: 0.7921 | Recall: 0.8121 | F1: 0.7942
Test  -- Loss: 1.0210 | Precision: 0.7524 | Recall: 0.7854 | F1: 0.7598
Time Elapsed: 0:02:29.946880, Time Remaining: 0:04:59.893760 

Epoch 149/150


100%|██████████| 15/15 [00:08<00:00,  1.71it/s]


Train -- Loss: 0.0273 | Precision: 0.9525 | Recall: 0.9518 | F1: 0.9509
Val   -- Loss: 0.2415 | Precision: 0.9196 | Recall: 0.9233 | F1: 0.9200
Devel -- Loss: 0.8452 | Precision: 0.7839 | Recall: 0.8148 | F1: 0.7921
Test  -- Loss: 1.0235 | Precision: 0.7492 | Recall: 0.7884 | F1: 0.7598
Time Elapsed: 0:02:30.086032, Time Remaining: 0:02:30.086032 

Epoch 150/150


100%|██████████| 15/15 [00:08<00:00,  1.68it/s]


Train -- Loss: 0.0272 | Precision: 0.9510 | Recall: 0.9506 | F1: 0.9495
Val   -- Loss: 0.2427 | Precision: 0.9235 | Recall: 0.9219 | F1: 0.9220
Devel -- Loss: 0.8518 | Precision: 0.7822 | Recall: 0.7994 | F1: 0.7849
Test  -- Loss: 1.0268 | Precision: 0.7468 | Recall: 0.7752 | F1: 0.7549
Time Elapsed: 0:02:30.451091, Time Remaining: 0:00:00 

Best Model: Epoch 3, Best Validation Loss: 0.6823
Path log file:  logs/training_log_BERT2_2023-12-07_09-52-24.txt
